In [57]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import time
import os
import argparse
## Load the model 
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import torch.nn.functional as F


In [2]:
model_conv = torchvision.models.resnet34(pretrained='imagenet')

In [2]:
def mixup_batch(inp_batch, alpha):
    """
    Applies mixup augementation to a batch
    :param input_batch: tensor with batchsize as first dim
    :param alpha: lamda drawn from beta(alpha+1, alpha)
    """
    inp_clone = inp_batch.clone()
    #getting batch size
    batchsize = inp_batch.size()[0]

    #permute a clone
    perm = np.random.permutation(batchsize)
    for i in range(batchsize):
        inp_clone[i] = inp_batch[perm[i]]
    #generating different lambda for each sample
    #Refernced from http://www.inference.vc/mixup-data-dependent-data-augmentation/
    lam = torch.Tensor(np.random.beta(alpha+1, alpha, batchsize))
    lam = lam.view(-1,1,1,1)
    inp_mixup = lam * inp_batch + (1- lam) * inp_clone
    return inp_mixup

In [3]:
def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, use_gpu, num_epochs=25, mixup = False, alpha = 0.1):
    print("MIXUP".format(mixup))
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0
    i=0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.eval() # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in tqdm(dataloaders[phase]):
                # get the inputs
                inputs, labels = data

                #augementation using mixup
                if phase == 'train' and mixup:
                    inputs = mixup_batch(inputs, alpha)

                if use_gpu:
                    inputs = inputs.cuda()
                    labels = labels.cuda()
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # # forward
                # outputs = model(inputs)
                # if type(outputs) == tuple:
                #     outputs, _ = outputs
                # _, preds = torch.max(outputs.data, 1)
                # loss = criterion(outputs, labels)

                with torch.set_grad_enabled(phase== 'train'):
                    outputs = model(inputs)
                    if type(outputs) == tuple:
                        outputs, _ = outputs
                    _,preds = torch.max(outputs,1)
                    loss=criterion(outputs,labels)
                # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.data.item()
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc >= best_acc :
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
            #model.load_state_dict(best_model_wts)
            if i%50==0:
                torch.save(model,ckptpth+str(i/10))
            i=i+1
            
        #print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [4]:
freeze_layers=1
n_class=5

In [5]:
model_conv

NameError: name 'model_conv' is not defined

In [30]:
## Lets freeze the first few layers. This is done in two stages 
# Stage-1 Freezing all the layers 
if freeze_layers:
  for i, param in model_conv.named_parameters():
    param.requires_grad = False

# Since imagenet as 1000 classes , We need to change our last layer according to the number of classes we have,
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, n_class)

In [31]:
# Stage-2 , freeze_layers all the layers till "layer2"
ct = []
for name, child in model_conv.named_children():
    if "layer2" in ct:
        for params in child.parameters():
            params.requires_grad = True
    ct.append(name)
    print(name)
    
# To view which layers are freeze and which layers are not freezed:
for name, child in model_conv.named_children():
    print(name)
    for name_2, params in child.named_parameters():
        print(name_2, params.requires_grad)

conv1
bn1
relu
maxpool
layer1
layer2
layer3
layer4
avgpool
fc
conv1
weight False
bn1
weight False
bias False
relu
maxpool
layer1
0.conv1.weight False
0.bn1.weight False
0.bn1.bias False
0.conv2.weight False
0.bn2.weight False
0.bn2.bias False
1.conv1.weight False
1.bn1.weight False
1.bn1.bias False
1.conv2.weight False
1.bn2.weight False
1.bn2.bias False
2.conv1.weight False
2.bn1.weight False
2.bn1.bias False
2.conv2.weight False
2.bn2.weight False
2.bn2.bias False
layer2
0.conv1.weight False
0.bn1.weight False
0.bn1.bias False
0.conv2.weight False
0.bn2.weight False
0.bn2.bias False
0.downsample.0.weight False
0.downsample.1.weight False
0.downsample.1.bias False
1.conv1.weight False
1.bn1.weight False
1.bn1.bias False
1.conv2.weight False
1.bn2.weight False
1.bn2.bias False
2.conv1.weight False
2.bn1.weight False
2.bn1.bias False
2.conv2.weight False
2.bn2.weight False
2.bn2.bias False
3.conv1.weight False
3.bn1.weight False
3.bn1.bias False
3.conv2.weight False
3.bn2.weight False
3

In [6]:
data_dir = "Datas/"
model_conv=torch.load("models_res/final")

In [7]:
    
## Loading the dataloaders -- Make sure that the data is saved in following way
"""
data/
  - train/
      - class_1 folder/
          - img1.png
          - img2.png
      - class_2 folder/
      .....
      - class_n folder/
  - val/
      - class_1 folder/
      - class_2 folder/
      ......
      - class_n folder/
"""
input_shape = 299
batch_size = 10
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]
scale = 360
input_shape = 224
use_parallel = True
use_gpu = True
epochs = 30

data_transforms = {
        'train': transforms.Compose([
        transforms.Resize(scale),
        transforms.RandomResizedCrop(input_shape),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(degrees=90),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)]),
        'val': transforms.Compose([
        transforms.Resize(scale),
        transforms.CenterCrop(input_shape),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)]),}


In [8]:
ckptpth="models_res/ckptk.model"

In [9]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                      data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                         shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [10]:
'''if use_parallel:
    print("[Using all the available GPUs]")
    model_conv = nn.DataParallel(model_conv, device_ids=[0, 1])
'''
print("[Using CrossEntropyLoss...]")
criterion = nn.CrossEntropyLoss()

print("[Using small learning rate with momentum...]")
optimizer_conv = optim.SGD(list(filter(lambda p: p.requires_grad, model_conv.parameters())), lr=0.001, momentum=0.9)

print("[Creating Learning rate scheduler...]")
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

print("[Training the model begun ....]")
#torch.cuda.empty_cache() 
model_ft = train_model(model_conv.cuda(), dataloaders, dataset_sizes, criterion, optimizer_conv, exp_lr_scheduler, 1,
                     num_epochs=epochs)

[Using CrossEntropyLoss...]
[Using small learning rate with momentum...]
[Creating Learning rate scheduler...]
[Training the model begun ....]
MIXUP
Epoch 0/29
----------


HBox(children=(IntProgress(value=0, max=142), HTML(value='')))


train Loss: 0.0408 Acc: 0.8647


HBox(children=(IntProgress(value=0, max=48), HTML(value='')))


val Loss: 0.0120 Acc: 0.9619
Epoch 1/29
----------


...


val Loss: 0.0110 Acc: 0.9619
Epoch 29/29
----------


HBox(children=(IntProgress(value=0, max=142), HTML(value='')))


train Loss: 0.0342 Acc: 0.8739


HBox(children=(IntProgress(value=0, max=48), HTML(value='')))


val Loss: 0.0114 Acc: 0.9576
Training complete in 6m 40s
Best val Acc: 0.968220


In [11]:
torch.save(model_ft,"models_res/coarse_finale_2layer_train_Epoch")

For_Fine_Grain CLasification

In [2]:
model=torch.load("models_res/coarse_finale_2layer_train_Epoch")

NameError: name 'torch' is not defined

In [69]:
## Lets freeze the first few layers. This is done in two stages 
# Stage-1 Freezing all the layers 
for i, param in model.named_parameters():
    param.requires_grad = False
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs,5)
#model = nn.Sequential(,models.resnet.BasicBlock(512,512),nn.AdaptiveAvgPool2d((1, 1)),nn.Linear(num_ftrs,36))
model.fc=nn.Sequential(nn.Linear(num_ftrs,out_features=2048),nn.Sigmoid(),nn.Linear(in_features=2048,out_features=2048),nn.Sigmoid(),nn.Linear(in_features=2048,out_features=36))

# Stage-2 , Freeze all the layers till "Conv2d_4a_3*3"
ct = []
# for name, child in model.named_children():
#     if "layer3" in ct:
#         for params in child.parameters():
#             params.requires_grad = True
#     ct.append(name)
#     print(name)
    
# To view which layers are freeze and which layers are not freezed:
for name, child in model.named_children():
    print(name)
    for name_2, params in child.named_parameters():
        print(name_2, params.requires_grad)

conv1
weight False
bn1
weight False
bias False
relu
maxpool
layer1
0.conv1.weight False
0.bn1.weight False
0.bn1.bias False
0.conv2.weight False
0.bn2.weight False
0.bn2.bias False
1.conv1.weight False
1.bn1.weight False
1.bn1.bias False
1.conv2.weight False
1.bn2.weight False
1.bn2.bias False
2.conv1.weight False
2.bn1.weight False
2.bn1.bias False
2.conv2.weight False
2.bn2.weight False
2.bn2.bias False
layer2
0.conv1.weight False
0.bn1.weight False
0.bn1.bias False
0.conv2.weight False
0.bn2.weight False
0.bn2.bias False
0.downsample.0.weight False
0.downsample.1.weight False
0.downsample.1.bias False
1.conv1.weight False
1.bn1.weight False
1.bn1.bias False
1.conv2.weight False
1.bn2.weight False
1.bn2.bias False
2.conv1.weight False
2.bn1.weight False
2.bn1.bias False
2.conv2.weight False
2.bn2.weight False
2.bn2.bias False
3.conv1.weight False
3.bn1.weight False
3.bn1.bias False
3.conv2.weight False
3.bn2.weight False
3.bn2.bias False
layer3
0.conv1.weight False
0.bn1.weight False

In [74]:
ckptpth="models_res/fin_cust_pretrain_finale_32_.001_mid"

data_dir = "Datas_1/"

In [45]:
input_shape = 299
batch_size = 35
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]
scale = 360
input_shape = 224 
use_parallel = True
use_gpu = True
epochs = 60
data_transforms = {
        'train': transforms.Compose([
        transforms.Resize(scale),
        transforms.RandomResizedCrop(input_shape),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(degrees=90),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)]),
        'val': transforms.Compose([
        transforms.Resize(scale),
        transforms.CenterCrop(input_shape),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)]),}
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                      data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                         shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f36b06f22e8>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 717, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 711, in _shutdown_workers
    q.close()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 134, in close
    self._reader.close()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor


In [23]:
'''if use_parallel:
    print("[Using all the available GPUs]")
    model_conv = nn.DataParallel(model_conv, device_ids=[0, 1])
'''
print("[Using CrossEntropyLoss...]")
criterion = nn.CrossEntropyLoss()

print("[Using small learning rate with momentum...]")
optimizer_conv = optim.SGD(list(filter(lambda p: p.requires_grad, model.parameters())), lr=0.0001, momentum=0.9)

print("[Creating Learning rate scheduler...]")
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=30, gamma=0.4)

print("[Training the model begun ....]")
modele = train_model(model.cuda(), dataloaders, dataset_sizes, criterion, optimizer_conv, exp_lr_scheduler, 1,
                     num_epochs=epochs)

[Using CrossEntropyLoss...]
[Using small learning rate with momentum...]
[Creating Learning rate scheduler...]
[Training the model begun ....]
MIXUP
Epoch 0/59
----------


HBox(children=(IntProgress(value=0, max=41), HTML(value='')))


train Loss: 0.0020 Acc: 0.9773


HBox(children=(IntProgress(value=0, max=14), HTML(value='')))


val Loss: 0.0004 Acc: 0.9958
Epoch 1/59
----------


...


val Loss: 0.0004 Acc: 0.9958
Epoch 23/59
----------


HBox(children=(IntProgress(value=0, max=41), HTML(value='')))


train Loss: 0.0022 Acc: 0.9731


HBox(children=(IntProgress(value=0, max=14), HTML(value='')))

KeyboardInterrupt: 

Best Acc=82.89/86.9 ReLU  Linear=85.68/89.05

In [24]:
x=model.fc

In [25]:
torch.save(x,"models_res_ext_fin/fin2coarse")

In [14]:
model=torch.load("models_res/coarse_finale_2layer_train_Epoch")

In [1]:
class MyModel():
    def __init__(self, modelp,fc):
        super(MyModel, self).__init__()
        self.inplanes = model.inplanes
        self.conv1 = modelp.conv1
        self.bn1 = modelp.bn1
        self.relu = modelp.relu
        self.maxpool = modelp.maxpool
        self.layer1 = modelp.layer1
        self.layer2 = modelp.layer2
        self.layer3 = modelp.layer3
        self.layer4 = modelp.layer4
        self.avgpool= modelp.avgpool # create layer
        self.fc1=modelp.fc
        self.fc2=fc
    def evaluate(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        y=self.fc2(x)
        x=self.fc1(x)
       
        return (x,y)

In [3]:
k=MyModel(model,x)


NameError: name 'model' is not defined

In [37]:
print("[Using CrossEntropyLoss...]")
criterion = nn.CrossEntropyLoss()

print("[Using small learning rate with momentum...]")
#optimizer_conv = optim.SGD(list(filter(lambda p: p.requires_grad, model2.parameters())), lr=0.001, momentum=0.9)

#print("[Creating Learning rate scheduler...]")
torch.set_grad_enabled(False)
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)
running_loss = 0.0
running_corrects=0 
for data in tqdm(dataloaders["val"]):
    # get the inputs
    inputs, labels = data
    inputs = inputs.cuda()
    labels = labels.cuda()
    # zero the parameter gradients
    optimizer_conv.zero_grad()
    
    outputs = model2(inputs)
    outputs=model.fc(outputs)
    if type(outputs) == tuple:
        outputs, _ = outputs
    _,preds = torch.max(outputs,1)
    loss=criterion(outputs,labels)
    # statistics
    running_loss += loss.data.item()
    running_corrects += torch.sum(preds == labels.data)

epoch_loss = running_loss / dataset_sizes["val"]
epoch_acc = running_corrects.double() / dataset_sizes["val"]

print('Loss: {:.4f} Acc: {:.4f}'.format( epoch_loss, epoch_acc))
torch.set_grad_enabled(True)

[Using CrossEntropyLoss...]
[Using small learning rate with momentum...]


HBox(children=(IntProgress(value=0, max=49), HTML(value='')))

RuntimeError: size mismatch, m1: [5120 x 1], m2: [512 x 5] at /pytorch/aten/src/THC/generic/THCTensorMathBlas.cu:266

In [ ]:
model2

In [30]:
data_dir="Datas_1"

In [31]:
    
## Loading the dataloaders -- Make sure that the data is saved in following way
"""
data/
  - train/
      - class_1 folder/
          - img1.png
          - img2.png
      - class_2 folder/
      .....
      - class_n folder/
  - val/
      - class_1 folder/
      - class_2 folder/
      ......
      - class_n folder/
"""
input_shape = 299
batch_size = 10
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]
scale = 360
input_shape = 224
use_parallel = True
use_gpu = True
epochs = 30

data_transforms = {
        'train': transforms.Compose([
        transforms.Resize(scale),
        transforms.RandomResizedCrop(input_shape),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(degrees=90),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)]),
        'val': transforms.Compose([
        transforms.Resize(scale),
        transforms.CenterCrop(input_shape),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)]),}


image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                      data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                         shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [32]:
class_names

['aircrafts_1',
 'aircrafts_2',
 'aircrafts_3',
 'aircrafts_4',
 'aircrafts_5',
 'aircrafts_6',
 'aircrafts_7',
 'birds_1',
 'birds_10',
 'birds_11',
 'birds_2',
 'birds_3',
 'birds_4',
 'birds_5',
 'birds_6',
 'birds_7',
 'birds_8',
 'birds_9',
 'cars_1',
 'cars_2',
 'cars_3',
 'cars_4',
 'cars_5',
 'cars_6',
 'cars_7',
 'cars_8',
 'dogs_1',
 'dogs_2',
 'dogs_3',
 'dogs_4',
 'dogs_5',
 'flowers_1',
 'flowers_2',
 'flowers_3',
 'flowers_4',
 'flowers_5']

In [53]:

print("[Using CrossEntropyLoss...]")
criterion = nn.CrossEntropyLoss()

print("[Using small learning rate with momentum...]")
optimizer_conv = optim.SGD(list(filter(lambda p: p.requires_grad, model2.parameters())), lr=0.001, momentum=0.9)

print("[Creating Learning rate scheduler...]")
torch.set_grad_enabled(False)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)
running_loss = 0.0
running_corrects=0 
for data in tqdm(dataloaders["val"]):
    # get the inputs
    inputs, labels = data
    inputs = inputs.cuda()
    labels = labels.cuda()
    # zero the parameter gradients
    optimizer_conv.zero_grad()
    
    outputs = model1(inputs)
    if type(outputs) == tuple:
        outputs, _ = outputs
    _,preds = torch.max(outputs,1)
    loss=criterion(outputs,labels)
    # statistics
    running_loss += loss.data.item()
    running_corrects += torch.sum(preds == labels.data)

epoch_loss = running_loss / dataset_sizes["val"]
epoch_acc = running_corrects.double() / dataset_sizes["val"]

print('Loss: {:.4f} Acc: {:.4f}'.format( epoch_loss, epoch_acc))
torch.set_grad_enabled(True)

In [35]:
class MyNet(nn.Module):
    def __init__(self):
        super(Net, self,classes,model).__init__()
        self.modeli=
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
model=Net()

SyntaxError: invalid syntax (<ipython-input-35-6c7d346e648e>, line 4)